In [1]:
from tensorflow import keras

2022-07-09 13:23:34.839686: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2022-07-09 13:23:34.839730: I tensorflow/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.


단어 임베딩을 만드는 방법은 두 가지이다.
* 관심 대상인 문제와 함게 단어 임베딩을 학습힌다. 이런 경우에는 랜덤한 단어 벡터로 시작해서 신경망의 가중치를 학습하는 것과 같은 방식으로 단어 벡터를 학습한다.
* 풀려는 문제가 아니고 다른 머신 러닝 작업에서 미리 계산된 단어 임베딩을 로드한다. 이를 사전 훈련된 단어 임베딩이라고 한다.

## `Embeeding` 층을 사용해 단어 임베딩 학습하기

In [2]:
from tensorflow.keras.layers import Embedding

# Embedding 층은 적어도 두 개의 매개변수를 받는다.
# 가능한 토큰의 개수 (여기서는 1000으로 단어 인덱스 최댓값 + 1)와 임베딩 차원(여기서는 64) 이다.
embedding_layer = Embedding(1000, 64)

`Embedding`층을 정수 인덱스를 밀집 벡터로 매핑하는 딕셔너리로 이해하는 것이 가장 좋다. 정수를 입력으로 받아 내부 딕셔너리에서 이 정수에 연관된 벡터를 찾아 반환한다.

`Embeding`층은 크기가 `(samples, sequence_length)`인 2D 정수 텐서를 입력으로 받는다. 각 샘플은 정수의 시퀀스이다. 가변 길이의 시퀀스를 임베딩할 수 있다. 배치에 있는 모든 시퀀스를 길이가 같아야 하므로 작은 길이의 시퀀스는 0으로 패딩되고 길이가 더 긴 시퀀스는 잘린다.

`Embedding`층은 크기가 `(samples, sequence_length, embedding_dim)`인 3d 실수형 텐서를 반환한다. 이런 3d 텐서는 rnn층이나 1d 합성곱 층에서 처리된다.  

IMDB 영화 리뷰 ㄱ마성 예측문제에 적용해 보자. 영화 리부에서 가장 빈도가 높은 1만 개의 단어를 추출하고 리뷰에서 20개가 넘는 단어는 버린다. 이 네트워크는 1만 개의 단어에 대해 8차원의 임베딩을 학습하여 정수 시퀀스 입력을 임베딩 시퀀스로 바꿀 것. 그 다음 이 텐서를 2d로 펼쳐서 분류를 위한 dense층을 훈련하겠다.

In [3]:
from tensorflow.keras.datasets import imdb
from tensorflow.keras import preprocessing

# 특성으로 사용할 단어의 수
max_features = 10000
# 사용할 텍스트의 길이 (가장 빈번한 max_features 개의 단어만 사용)
maxlen = 20

# 정수 리스트로 데이터를 로드한다.
(x_train, y_train), (x_test, y_test) = imdb.load_data(num_words=max_features)

# 리스트를 (samples, maxlen) 크기의 2d 정수 텐서로 반환
x_train = preprocessing.sequence.pad_sequences(x_train, maxlen=maxlen)
x_test = preprocessing.sequence.pad_sequences(x_test, maxlen=maxlen)

17473536/17464789 [==============================] - 1s 0us/step


In [5]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten, Dense, Embedding

model  = Sequential()
# 나중에 임베딩된 입력을 Flatten 층에서 펼치기 위해 Embedding 층에 input_length를 지정한다.
model.add(Embedding(10000, 8, input_length=maxlen))
# Embedding 층의 츨력 크기는 (samples, maxlen, 8) 크기로 2d 텐서로 펼친다.

# 3D 임베딩 텐서를 (samples, maxlen*8)크기의 2d 텐서로 펼친다.
model.add(Flatten())

# 분류기를 추가한다.
model.add(Dense(1, activation='sigmoid'))
model.compile(optimizer='Adam', loss='binary_crossentropy', metrics=['accuracy'])
model.summary()

history = model.fit(x_train, y_train,
                   epochs=10,
                   batch_size=32,
                   validation_split=0.2)

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 8)             80000     
_________________________________________________________________
flatten (Flatten)            (None, 160)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 161       
Total params: 80,161
Trainable params: 80,161
Non-trainable params: 0
_________________________________________________________________


/home/ubuntu/.pyenv/versions/3.8.12/lib/python3.8/site-packages/pandas/compat/__init__.py:124: UserWarning: Could not import the lzma module. Your installed Python is incomplete. Attempting to use lzma compression will result in a RuntimeError.
  warnings.warn(msg)
2022-07-09 14:05:14.684986: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


Epoch 1/10
625/625 [==============================] - 1s 1ms/step - loss: 0.6442 - accuracy: 0.6575 - val_loss: 0.5574 - val_accuracy: 0.7264
Epoch 2/10
625/625 [==============================] - 1s 923us/step - loss: 0.4718 - accuracy: 0.7915 - val_loss: 0.4928 - val_accuracy: 0.7510
Epoch 3/10
625/625 [==============================] - 1s 926us/step - loss: 0.3814 - accuracy: 0.8416 - val_loss: 0.4881 - val_accuracy: 0.7562
Epoch 4/10
625/625 [==============================] - 1s 1ms/step - loss: 0.3176 - accuracy: 0.8773 - val_loss: 0.5002 - val_accuracy: 0.7504
Epoch 5/10
625/625 [==============================] - 1s 953us/step - loss: 0.2637 - accuracy: 0.9066 - val_loss: 0.5186 - val_accuracy: 0.7490
Epoch 6/10
625/625 [==============================] - 1s 994us/step - loss: 0.2172 - accuracy: 0.9288 - val_loss: 0.5428 - val_accuracy: 0.7448
Epoch 7/10
625/625 [==============================] - 1s 980us/step - loss: 0.1783 - accuracy: 0.9467 - val_loss: 0.5729 - val_accuracy: 0.7

## 사전 훈련된 단어 임베딩 사용하기

훈련데이터가 부족하면 작업에 맞는 단어 임베딩을 학습할 수 없다.  
풀려는 문제와 함께 단어 임베딩을 학습하는 대신에 미리 계산된 임베딩 공간에서 임베딩 벡터를 로드할 수 있다. 이런 임베딩 공간은 뛰어난 구조와 유용한 성질을 가지고 있어서 언어 구조의 일반적인 측면을 잡아낼 수 있다.  


In [6]:
import os

imdb_dir = './aclImdb/'
train_dir = os.path.join(imdb_dir, 'train')

labels = []
texts = []

for label_type in ['neg', 'pos']:
    dir_name = os.path.join(train_dir, label_type)
    for fname in os.listdir(dir_name):
        if fname[-4:] == '.txt':
            f = open(os.path.join(dir_name, fname), encoding='utf8')
            texts.append(f.read())
            f.close()
            if label_type == 'neg':
                labels.append(0)
            else:
                labels.append(1)

### 데이터 토큰화

In [7]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
import numpy as np

maxlen = 100  # 100개 단어 이후는 버립니다
training_samples = 200  # 훈련 샘플은 200개입니다
validation_samples = 10000  # 검증 샘플은 10,000개입니다
max_words = 10000  # 데이터셋에서 가장 빈도 높은 10,000개의 단어만 사용합니다

tokenizer = Tokenizer(num_words=max_words)
tokenizer.fit_on_texts(texts)
sequences = tokenizer.texts_to_sequences(texts)

word_index = tokenizer.word_index
print('%s개의 고유한 토큰을 찾았습니다.' % len(word_index))

data = pad_sequences(sequences, maxlen=maxlen)

labels = np.asarray(labels)
print('데이터 텐서의 크기:', data.shape)
print('레이블 텐서의 크기:', labels.shape)

# 데이터를 훈련 세트와 검증 세트로 분할합니다.
# 샘플이 순서대로 있기 때문에 (부정 샘플이 모두 나온 후에 긍정 샘플이 옵니다) 
# 먼저 데이터를 섞습니다.
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
labels = labels[indices]

x_train = data[:training_samples]
y_train = labels[:training_samples]
x_val = data[training_samples: training_samples + validation_samples]
y_val = labels[training_samples: training_samples + validation_samples]

22977개의 고유한 토큰을 찾았습니다.
데이터 텐서의 크기: (1787, 100)
레이블 텐서의 크기: (1787,)


## GloVe 단어임베딩 내려받기